In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objs as go
import plotly.express as px
import joblib
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout,GRU,RNN,Conv1D,SimpleRNN, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers, metrics, losses
from tensorflow.keras.callbacks import EarlyStopping




In [3]:
data = pd.read_csv('3.csv')

In [4]:
scaler = MinMaxScaler()
# Get the index of the label column
label_column = "Total Cloud Cover [%]"
label_column_index = data.columns.get_loc(label_column)

# Determine the split point (e.g., 80% training, 20% testing)
split_point = int(len(data) * 0.9)

# Split the data into training and testing sets
train_data = data.iloc[:split_point]
test_data = data.iloc[split_point:]

train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.transform(test_data)

In [5]:
# Convert the DataFrames to NumPy arrays
train_data_scaled = np.asarray(train_data_scaled).astype(np.float32)
test_data_scaled = np.asarray(test_data_scaled).astype(np.float32)

class TimeSeriesGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, look_back=75, step_ahead=30, batch_size=32, shuffle=True, target_column_index=-1):
        'Initialization'
        self.data = data
        self.look_back = look_back
        self.step_ahead = step_ahead
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.target_column_index = target_column_index
        self.indices = np.arange(len(data) - look_back - step_ahead + 1)
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        # Generate data
        X, y = self.__data_generation(batch_indices)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __data_generation(self, batch_indices):
        'Generates data containing batch_size samples'
        X = np.empty((self.batch_size, self.look_back, self.data.shape[1]))
        y = np.empty((self.batch_size,))
        
        for i, idx in enumerate(batch_indices):
            X[i,] = self.data[idx:idx + self.look_back]
            y[i,] = self.data[idx + self.look_back + self.step_ahead - 1, self.target_column_index]
                

        return X, y

In [6]:
def weighted_rmse(weights_zero, weights_nonzero):
    def loss(y_true, y_pred):
        # Create a mask for zero and non-zero values
        is_zero = tf.cast(tf.equal(y_true, 0), tf.float32)
        is_nonzero = 1 - is_zero
        # Calculate squared error
        sq_error = tf.square(y_true - y_pred)
        
        # Apply different weights to zero and non-zero errors
        weighted_sq_error = (is_zero * weights_zero + is_nonzero * weights_nonzero) * sq_error
        
        # Calculate mean of weighted squared error and then take the square root
        mean_weighted_sq_error = tf.reduce_mean(weighted_sq_error)
        rmse = tf.sqrt(mean_weighted_sq_error)
        return rmse
    
    return loss

In [7]:
# Define configurations
lstm_layer_configs =  [[100]]
dense_layer_configs =  [[32,64]]
configurations = [(lstm, dense) for lstm in lstm_layer_configs for dense in dense_layer_configs]

In [8]:
def build_model_bilstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape):
    model = Sequential()
    for i in range(num_lstm_layers):
        if i == 0:
            # First LSTM layer with input shape
            model.add(Bidirectional(LSTM(lstm_units[i], return_sequences=(i < num_lstm_layers - 1), input_shape=input_shape)))
            model.add(Dropout(0.2))
        else:
            model.add(Bidirectional(LSTM(lstm_units[i], return_sequences=(i < num_lstm_layers - 1))))

    for i in range(num_dense_layers):
        model.add(Dense(dense_units[i], activation='relu'))
  
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(), loss=weighted_rmse(weights_zero=0.2, weights_nonzero=0.8), metrics=[metrics.RootMeanSquaredError(), 'mae'])
    return model

In [9]:
def build_model_convlstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape):
    model = Sequential()
    # Add Conv1D layer
    model.add(Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape))
    
    for i in range(num_lstm_layers):
        if i == 0:
            # First LSTM layer with input shape
            model.add(LSTM(lstm_units[i], return_sequences=(i < num_lstm_layers - 1)))
            model.add(Dropout(0.2))
        else:
            model.add(LSTM(lstm_units[i], return_sequences=(i < num_lstm_layers - 1)))

    for i in range(num_dense_layers):
        model.add(Dense(dense_units[i], activation='relu'))
  
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(), loss=weighted_rmse(weights_zero=0.2, weights_nonzero=0.8), metrics=[metrics.RootMeanSquaredError(), 'mae'])
    return model

In [10]:
def run_model(model_type, lstm_units, dense_units, input_shape, train_gen, test_gen, horizon):
    num_lstm_layers = len(lstm_units)
    num_dense_layers = len(dense_units)
    print(f"Building model with {num_lstm_layers} {model_type.upper()} layers and {num_dense_layers} Dense layers for {horizon}-minute horizon")
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    if model_type == 'lstm':
        model = build_model_lstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    elif model_type == 'rnn':
        model = build_model_rnn(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    elif model_type == 'gru':
        model = build_model_gru(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    elif model_type == 'bilstm':
        model = build_model_bilstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    elif model_type == 'convlstm':
        model = build_model_convlstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    
    print(model.summary())

    # Train the model
    history = model.fit(train_gen, epochs=10, validation_data=test_gen, callbacks=[early_stopping])

    # Evaluate the model on the scaled test data
    test_loss, test_rmse, test_mae = model.evaluate(test_gen)
    print(f'Test Loss (scaled): {test_loss}, Test RMSE (scaled): {test_rmse}, Test MAE (scaled): {test_mae}')

    # Make predictions on the test data with a progress bar
    predictions = []
    true_labels = []
    for i in tqdm(range(len(test_gen)), desc='Predicting', unit='batch'):
        x_test, y_test = test_gen[i]
        pred = model.predict(x_test, verbose=0)
        predictions.append(pred)
        true_labels.append(y_test)
    
    testPredict = np.concatenate(predictions).reshape(-1, 1)
    testY = np.concatenate(true_labels).reshape(-1, 1)

    # Repeat predictions to match the shape of the original data
    testPredict_repeated = np.repeat(testPredict, 19, axis=-1)
    testY_repeated = np.repeat(testY, 19, axis=-1)

    # Inverse transform the repeated arrays
    testPredict_original = scaler.inverse_transform(testPredict_repeated)[:, label_column_index]
    testY_original = scaler.inverse_transform(testY_repeated)[:, label_column_index]

    # Calculate evaluation metrics on the original scale
    test_rmse_original = math.sqrt(mean_squared_error(testY_original, testPredict_original))
    test_mae_original = mean_absolute_error(testY_original, testPredict_original)
    test_r2_original = r2_score(testY_original, testPredict_original)
    
    print(f'Test RMSE (original scale): {test_rmse_original}')
    print(f'Test MAE (original scale): {test_mae_original}')
    print(f'Test R² (original scale): {test_r2_original}')

    return history, test_loss, test_rmse_original, test_mae_original, test_r2_original, testY_original, testPredict_original


In [11]:
forecast_horizons = [30, 25, 15] 
models = {}
weights = {15: 0.5, 25: 0.35, 30: 0.15}
model_type = 'bilstm'

all_predictions = {}
all_true_values = {}

for horizon in forecast_horizons:
    print(f"Training model for {horizon}-minute ahead prediction:")
    # Create generators for each horizon
    train_gen = TimeSeriesGenerator(train_data_scaled, step_ahead=horizon, batch_size=32, shuffle=True, target_column_index=6)
    test_gen = TimeSeriesGenerator(test_data_scaled, step_ahead=horizon, batch_size=32, shuffle=False, target_column_index=6)
    input_shape = (train_gen.look_back, train_data_scaled.shape[1])
    
    config_index = 0  # Adjust this index to run different configurations
    lstm_units, dense_units = configurations[config_index]
    num_lstm_layers = len(lstm_units)
    num_dense_layers = len(dense_units)

    model = build_model_bilstm(num_lstm_layers, lstm_units, num_dense_layers, dense_units, input_shape)
    print(model.summary())
    
    history, test_loss, test_rmse, test_mae, test_r2_original, testY_original, testPredict_original = run_model(model_type, lstm_units, dense_units, input_shape, train_gen, test_gen, horizon)
    models[horizon] = (history, test_loss, test_rmse, test_mae, test_r2_original)
    
    all_true_values[horizon] = testY_original
    all_predictions[horizon] = testPredict_original

weighted_sum = sum(weights[horizon] * models[horizon][4] for horizon in forecast_horizons)
print(f"Weighted Sum of R² Scores: {weighted_sum}")

# Plotting all horizons on a single graph
fig = go.Figure()

for horizon in forecast_horizons:
    fig.add_trace(go.Scatter(x=list(range(len(all_true_values[horizon]))), y=all_true_values[horizon], mode='lines', name=f'True Values {horizon}-min'))
    fig.add_trace(go.Scatter(x=list(range(len(all_predictions[horizon]))), y=all_predictions[horizon], mode='lines', name=f'Predicted Values {horizon}-min'))

fig.update_layout(title='True vs Predicted Values for All Horizons', xaxis_title='Index', yaxis_title='Value', template='plotly_dark')

# Save the interactive plot as an HTML file
filename = "forecast_all_horizons_bilstm.html"
fig.write_html(filename)
print(f"Interactive plot saved as {filename}")

fig.show()



Training model for 30-minute ahead prediction:


ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
forecast_horizons = [30, 25, 15] 
models = {}
weights = {15: 0.5, 25: 0.35, 30: 0.15}
model_type = 'convlstm'

all_predictions = {}
all_true_values = {}

for horizon in forecast_horizons:
    print(f"Training model for {horizon}-minute ahead prediction:")
    # Create generators for each horizon
    train_gen = TimeSeriesGenerator(train_data_scaled, step_ahead=horizon, batch_size=32, shuffle=True, target_column_index=6)
    test_gen = TimeSeriesGenerator(test_data_scaled, step_ahead=horizon, batch_size=32, shuffle=False, target_column_index=6)
    input_shape = (train_gen.look_back, train_data_scaled.shape[1])
    
    config_index = 0  # Adjust this index to run different configurations
    lstm_units, dense_units = configurations[config_index]
    
    history, test_loss, test_rmse, test_mae, test_r2_original, testY_original, testPredict_original = run_model(model_type, lstm_units, dense_units, input_shape, train_gen, test_gen, horizon)
    models[horizon] = (history, test_loss, test_rmse, test_mae, test_r2_original)
    
    all_true_values[horizon] = testY_original
    all_predictions[horizon] = testPredict_original

weighted_sum = sum(weights[horizon] * models[horizon][4] for horizon in forecast_horizons)
print(f"Weighted Sum of R² Scores: {weighted_sum}")

# Plotting all horizons on a single graph
fig = go.Figure()

for horizon in forecast_horizons:
    fig.add_trace(go.Scatter(x=list(range(len(all_true_values[horizon]))), y=all_true_values[horizon], mode='lines', name=f'True Values {horizon}-min'))
    fig.add_trace(go.Scatter(x=list(range(len(all_predictions[horizon]))), y=all_predictions[horizon], mode='lines', name=f'Predicted Values {horizon}-min'))

fig.update_layout(title='True vs Predicted Values for All Horizons', xaxis_title='Index', yaxis_title='Value', template='plotly_dark')

# Save the interactive plot as an HTML file
filename = "forecast_all_horizons_convlstm.html"
fig.write_html(filename)
print(f"Interactive plot saved as {filename}")

fig.show()



Training model for 30-minute ahead prediction:
Building model with 1 CONVLSTM layers and 2 Dense layers for 30-minute horizon
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 71, 32)            3072      
                                                                 
 lstm_1 (LSTM)               (None, 100)               53200     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 32)                3232      
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 1)                 65   